In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import random
import time
#PATH PROCESS
import os
import os.path
from pathlib import Path
import glob
from scipy.io import loadmat
import csv
#IMAGE PROCESS
from PIL import Image
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.preprocessing import image
from skimage.feature import hessian_matrix, hessian_matrix_eigvals
from scipy.ndimage.filters import convolve
from skimage import data, io, filters
import skimage
from skimage.morphology import convex_hull_image, erosion
from IPython import display
from scipy.ndimage import gaussian_filter
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.patches as patches
#SCALER & TRANSFORMATION
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from sklearn.preprocessing import LabelEncoder
#ACCURACY CONTROL
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
#OPTIMIZER
from keras.optimizers import RMSprop,Adam,Optimizer,Optimizer, SGD
#MODEL LAYERS
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN,\
LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Convolution2D, ZeroPadding2D,Reshape, Conv2DTranspose,\
LeakyReLU, GaussianNoise, GlobalMaxPooling2D, ReLU, Input, Concatenate
from keras import models
from keras import layers
import tensorflow as tf
from keras.applications import VGG16,VGG19,inception_v3
from keras import backend as K
from keras.utils import plot_model
from keras.datasets import mnist
import keras
from keras.models import Model
#IGNORING WARNINGS


/tmp/ipython-input-1907337387.py:23: DeprecationWarning: Please import `convolve` from the `scipy.ndimage` namespace; the `scipy.ndimage.filters` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.filters import convolve


To load data from Google Drive, you first need to mount your Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


After mounting your Google Drive, you can access your files using the path `/content/drive/My Drive/`. For example, if you have a file named `my_data.csv` in the root of your Drive, you can load it using pandas like this:

In [ ]:
path = "/content/drive/MyDrive/SDOBenchmark-data-example/training"
Main_Path = Path(path)
print(Main_Path)

/content/drive/MyDrive/SDOBenchmark-data-example/training


In [ ]:
print(os.listdir(path))

['meta_data.csv', '11389', '11388', '11391', '11390', '11392', '11393', '11394', '11401', '11405', '11399', '11410', '11403', '11395', '11408', '11409', '11412', '11413', '11428', '11430', '11432', '11422', '11429', '11436', '11433', '11434', '11442', '11438', '11445', '11459', '11460', '11452', '11455', '11453', '11458', '11448', '11464', '11461', '11465', '11469', '11472', '11479', '11481', '11471', '11466', '11486', '11490', '11491', '11493', '11492', '11497', '11506', '11499', '11508', '11504', '11505', '11498', '11494', '11510', '11511', '11512', '11513', '11515', '11520', '11514', '11522', '11521', '11518', '11525', '11519', '11528', '11530', '11532', '11541', '11533', '11535', '11542', '11538', '11546', '11547', '11543', '11558', '11561', '11560', '11569', '11573', '11562', '11575', '11566', '11564', '11579', '11589', '11591', '11610', '11590', '11596', '11612', '11613', '11614', '11615', '11593', '11600', '11623', '11629', '11632', '11633', '11652', '11638', '11631', '11642', '

In [ ]:
filepath = pd.Series(list(Main_Path.glob(r'**/*_magnetogram.jpg')), name = 'Filepath')

In [ ]:
os.path.split(filepath.values[1])

('/content/drive/MyDrive/SDOBenchmark-data-example/training/11389/2012_01_01_19_06_00_0',
 '2012-01-01T173600__magnetogram.jpg')

In [ ]:
import pandas as pd
import os
from datetime import datetime
import IPython.display

def calculate_time_difference(full_path):
    # Separate the directory and the filename
    directory_path, filename = os.path.split(full_path)

    # Extract the timestamp from the directory name
    dir_timestamp_str_raw = os.path.basename(directory_path)
    dir_timestamp_str_parsed = '_'.join(dir_timestamp_str_raw.split('_')[:6])

    # Convert the directory timestamp string to a datetime object
    dir_dt = datetime.strptime(dir_timestamp_str_parsed, '%Y_%m_%d_%H_%M_%S')


    file_timestamp_str = filename.split('__')[0]

    file_dt = datetime.strptime(file_timestamp_str, '%Y-%m-%dT%H%M%S')

    time_difference = file_dt - dir_dt
    time_difference_minutes = abs(time_difference.total_seconds()) / 60

    return dir_dt, file_dt, time_difference_minutes

results = filepath.apply(calculate_time_difference)

time_diff_df = pd.DataFrame(results.tolist(), columns=['Directory_Timestamp', 'File_Timestamp', 'Time_Difference_Minutes'])

time_diff_df['Filepath'] = filepath

IPython.display.display(time_diff_df.head())

# statistics
IPython.display.display(time_diff_df.describe())
# 10  90 300 720

,Directory_Timestamp,File_Timestamp,Time_Difference_Minutes,Filepath
0,2012-01-01 19:06:00,2012-01-01 14:06:00,300.0,/content/drive/MyDrive/SDOBenchmark-data-examp...
1,2012-01-01 19:06:00,2012-01-01 17:36:00,90.0,/content/drive/MyDrive/SDOBenchmark-data-examp...
2,2012-01-01 19:06:00,2012-01-01 18:56:00,10.0,/content/drive/MyDrive/SDOBenchmark-data-examp...
3,2012-01-01 19:06:00,2012-01-01 07:06:00,720.0,/content/drive/MyDrive/SDOBenchmark-data-examp...
4,2012-01-07 02:27:01,2012-01-06 14:27:01,720.0,/content/drive/MyDrive/SDOBenchmark-data-examp...


,Directory_Timestamp,File_Timestamp,Time_Difference_Minutes
count,3125,3125,3125.000000
mean,2014-05-20 07:12:09.648640,2014-05-20 19:43:08.643520,1088.986448
min,2012-01-01 19:06:00,2012-01-01 07:06:00,0.000000
25%,2013-01-20 17:38:20,2013-01-20 05:38:20,90.000000
50%,2014-04-18 11:54:00,2014-04-19 08:38:01,709.983333
75%,2015-07-08 11:10:00,2015-07-09 06:30:38,1243.000000
max,2017-12-23 06:30:50,2017-12-23 20:26:00,12229.983333
std,NaN,NaN,1611.143956


In [ ]:
time_diff_df

,Directory_Timestamp,File_Timestamp,Time_Difference_Minutes,Filepath
0,2012-01-01 19:06:00,2012-01-01 14:06:00,300.000000,/content/drive/MyDrive/SDOBenchmark-data-examp...
1,2012-01-01 19:06:00,2012-01-01 17:36:00,90.000000,/content/drive/MyDrive/SDOBenchmark-data-examp...
2,2012-01-01 19:06:00,2012-01-01 18:56:00,10.000000,/content/drive/MyDrive/SDOBenchmark-data-examp...
3,2012-01-01 19:06:00,2012-01-01 07:06:00,720.000000,/content/drive/MyDrive/SDOBenchmark-data-examp...
4,2012-01-07 02:27:01,2012-01-06 14:27:01,720.000000,/content/drive/MyDrive/SDOBenchmark-data-examp...
...,...,...,...,...
3120,2017-12-22 06:29:01,2017-12-22 01:29:01,300.000000,/content/drive/MyDrive/SDOBenchmark-data-examp...
3121,2017-12-11 12:00:00,2017-12-12 04:33:37,993.616667,/content/drive/MyDrive/SDOBenchmark-data-examp...
3122,2017-12-11 12:00:00,2017-12-12 03:13:37,913.616667,/content/drive/MyDrive/SDOBenchmark-data-examp...
3123,2017-12-11 12:00:00,2017-12-11 16:43:37,283.616667,/content/drive/MyDrive/SDOBenchmark-data-examp...


In [ ]:

time_diff_filtered_df = time_diff_df[(time_diff_df['Time_Difference_Minutes'] >= 8) & (time_diff_df['Time_Difference_Minutes'] <= 1440)]
IPython.display.display(time_diff_filtered_df.head())

,Directory_Timestamp,File_Timestamp,Time_Difference_Minutes,Filepath
0,2012-01-01 19:06:00,2012-01-01 14:06:00,300.0,/content/drive/MyDrive/SDOBenchmark-data-examp...
1,2012-01-01 19:06:00,2012-01-01 17:36:00,90.0,/content/drive/MyDrive/SDOBenchmark-data-examp...
2,2012-01-01 19:06:00,2012-01-01 18:56:00,10.0,/content/drive/MyDrive/SDOBenchmark-data-examp...
3,2012-01-01 19:06:00,2012-01-01 07:06:00,720.0,/content/drive/MyDrive/SDOBenchmark-data-examp...
4,2012-01-07 02:27:01,2012-01-06 14:27:01,720.0,/content/drive/MyDrive/SDOBenchmark-data-examp...


In [ ]:
time_diff_filtered_df

,Directory_Timestamp,File_Timestamp,Time_Difference_Minutes,Filepath
0,2012-01-01 19:06:00,2012-01-01 14:06:00,300.000000,/content/drive/MyDrive/SDOBenchmark-data-examp...
1,2012-01-01 19:06:00,2012-01-01 17:36:00,90.000000,/content/drive/MyDrive/SDOBenchmark-data-examp...
2,2012-01-01 19:06:00,2012-01-01 18:56:00,10.000000,/content/drive/MyDrive/SDOBenchmark-data-examp...
3,2012-01-01 19:06:00,2012-01-01 07:06:00,720.000000,/content/drive/MyDrive/SDOBenchmark-data-examp...
4,2012-01-07 02:27:01,2012-01-06 14:27:01,720.000000,/content/drive/MyDrive/SDOBenchmark-data-examp...
...,...,...,...,...
3120,2017-12-22 06:29:01,2017-12-22 01:29:01,300.000000,/content/drive/MyDrive/SDOBenchmark-data-examp...
3121,2017-12-11 12:00:00,2017-12-12 04:33:37,993.616667,/content/drive/MyDrive/SDOBenchmark-data-examp...
3122,2017-12-11 12:00:00,2017-12-12 03:13:37,913.616667,/content/drive/MyDrive/SDOBenchmark-data-examp...
3123,2017-12-11 12:00:00,2017-12-11 16:43:37,283.616667,/content/drive/MyDrive/SDOBenchmark-data-examp...


In [ ]:
df = time_diff_filtered_df[['Filepath', 'Time_Difference_Minutes']].copy()
df.rename(columns={'Filepath': 'path', 'Time_Difference_Minutes': 'time'}, inplace=True)


In [ ]:
data_df = df.sample(n=len(df), random_state=1).reset_index(drop=True)

train_val_df, test_df = train_test_split(data_df, test_size=0.1, random_state=0)

train_df, val_df = train_test_split(train_val_df, test_size=0.2, random_state=0) # 80% train, 20% val from train_val_df

train_df['path'] = train_df['path'].astype(str)
val_df['path'] = val_df['path'].astype(str)
test_df['path'] = test_df['path'].astype(str)

In [ ]:
test_df

,path,time
1732,/content/drive/MyDrive/Colab_Notebooks/PI-p/SD...,1312.633333
1451,/content/drive/MyDrive/Colab_Notebooks/PI-p/SD...,300.000000
1047,/content/drive/MyDrive/Colab_Notebooks/PI-p/SD...,10.000000
2391,/content/drive/MyDrive/Colab_Notebooks/PI-p/SD...,90.000000
2284,/content/drive/MyDrive/Colab_Notebooks/PI-p/SD...,90.000000
...,...,...
69,/content/drive/MyDrive/Colab_Notebooks/PI-p/SD...,1139.983333
1432,/content/drive/MyDrive/Colab_Notebooks/PI-p/SD...,10.000000
2123,/content/drive/MyDrive/Colab_Notebooks/PI-p/SD...,10.000000
1591,/content/drive/MyDrive/Colab_Notebooks/PI-p/SD...,720.000000


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

train_df['time_scaled'] = scaler.fit_transform(train_df[['time']])
val_df['time_scaled'] = scaler.transform(val_df[['time']])
test_df['time_scaled'] = scaler.transform(test_df[['time']])

Loading images

In [ ]:
def process_magnetogram(img):

    img = img / 255.0
    # Shift to -1 to 1 (approximate, assuming 127 was neutral)
    img = (img - 0.5) * 2.0
    return img

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=process_magnetogram, # Use this instead of rescale
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=process_magnetogram
)

In [ ]:
from numpy.random import shuffle
train_images = train_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'path',
    y_col = 'time_scaled',
    target_size = (64, 64),
    color_mode = 'grayscale',
    class_mode = 'raw',
    batch_size = 128,
    shuffle = True,
    seed = 42,
    subset = 'training',
    workers=8,  # Increased workers for parallel data processing
    use_multiprocessing=True # Enable multiprocessing
)

val_images = train_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'path',
    y_col = 'time_scaled',
    target_size = (64, 64),
    color_mode = 'grayscale',
    class_mode = 'raw',
    batch_size = 128,
    shuffle = True,
    seed = 42,
    subset = 'validation',
    workers=8, # Increased workers for parallel data processing
    use_multiprocessing=True # Enable multiprocessing
)

test_images = test_generator.flow_from_dataframe(
    dataframe = test_df,
    x_col = 'path',
    y_col = 'time_scaled',
    target_size = (64, 64),
    color_mode = 'grayscale',
    class_mode = 'raw',
    batch_size = 128,
    shuffle = False,
    workers=8, # Increased workers for parallel data processing
    use_multiprocessing=True # Enable multiprocessing
)

Found 1434 validated image filenames.
Found 358 validated image filenames.
Found 250 validated image filenames.


Regresion CNN buid

In [ ]:

inputs = tf.keras.Input(shape=(64, 64, 1))

#Layer 1
x = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), padding='same', kernel_initializer='he_normal')(inputs)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.LeakyReLU(alpha=0.1)(x)
x = tf.keras.layers.MaxPooling2D()(x)
#Layer 2
x = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), padding='same', kernel_initializer='he_normal')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.LeakyReLU(alpha=0.1)(x)
x = tf.keras.layers.MaxPooling2D()(x)
#Layer 3
x = tf.keras.layers.Conv2D(128, kernel_size=(3, 3), padding='same', kernel_initializer='he_normal')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.LeakyReLU(alpha=0.1)(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
# REGRESSION HEAD
x = tf.keras.layers.Dense(64)(x)
x = tf.keras.layers.LeakyReLU(alpha=0.1)(x)
x = tf.keras.layers.Dropout(0.2)(x)

outputs = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)
model.summary()

# 5. CALLBACKS


/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 64, 64, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 64, 64, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,889 (398.00 KB)

 Trainable params: 101,441 (396.25 KB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

# 6. TRAINING
history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[early_stopping],
    batch_size=32
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


KeyboardInterrupt: 

In [ ]:
model_path_in_drive = '/content/drive/MyDrive/PI-p/regression_model_V1.h5'
model.save(model_path_in_drive)
print(f"Model saved as '{model_path_in_drive}'")


Model saved as '/content/drive/MyDrive/Colab_Notebooks/PI-p/regression_model_V1.h5'


In [ ]:
from PIL import Image
import numpy as np

# Select a sample image from the test_df
sample_image_path = test_df.iloc[11]['path']
actual_time = test_df.iloc[11]['time']

print(f"Using image: {sample_image_path}")
print(f"Actual time for this image: {actual_time:.2f} minutes")

# Load and preprocess the image
image = Image.open(sample_image_path)
image = image.resize((64, 64))
image_array = np.array(image) / 255.0

image_array = np.expand_dims(image_array, axis=-1)
image_array = np.expand_dims(image_array, axis=0)

# Make prediction
prediction_scaled = model.predict(image_array)

# Inverse transform the scaled prediction
prediction_minutes = scaler.inverse_transform(prediction_scaled)

print(f"\nPredicted time for the image: {prediction_minutes[0][0]:.2f} minutes")

Using image: /content/drive/MyDrive/Colab_Notebooks/PI-p/SDOBenchmark-data-example/training/12205/2014_11_13_15_08_01_0/2014-11-13T145801__magnetogram.jpg
Actual time for this image: 10.00 minutes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step

Predicted time for the image: 676.87 minutes


In [ ]:
# Make predictions on the test set
predictions_scaled = loaded_model.predict(test_images)

predictions_minutes = scaler.inverse_transform(predictions_scaled)

test_df['predicted_time'] = predictions_minutes

actual_minutes = test_df['time'].values
predicted_minutes = test_df['predicted_time'].values

# MSE
mse = mean_squared_error(actual_minutes, predicted_minutes)
rmse = np.sqrt(mse)

print(f"Mean Squared Error (MSE) on the test set: {mse:.2f}")
print(f"Root Mean Squared Error (RMSE) on the test set: {rmse:.2f}")

3/4 ━━━━━━━━━━━━━━━━━━━━ 0s 622ms/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 900ms/step
Mean Squared Error (MSE) on the test set: 183885.80
Root Mean Squared Error (RMSE) on the test set: 428.82


In [ ]:
#load trained model
model_path_in_drive = '/content/drive/MyDrive/Colab_Notebooks/PI-p/regression_model_V1.h5'

loaded_model = tf.keras.models.load_model(model_path_in_drive)

print(f"Model loaded successfully from '{model_path_in_drive}'!")

Model loaded successfully from '/content/drive/MyDrive/Colab_Notebooks/PI-p/regression_model_V1.h5'!
